First import the dataset

In [1]:
#Import data handling
import pandas as pd

cancerData = pd.read_csv("../Dataset/lncRNA_5_Cancers.csv")

In [4]:
print(cancerData.shape)

(2529, 12311)


Our dataset consists of 2529 samples (rows) and 12311 features (columns).

The first row is for column label, and two of the columns are Ensembl_ID (first col) and Class (last col).

In [6]:
cancerData['Class'].unique()

array(['KIRC', 'LUAD', 'LUSC', 'PRAD', 'THCA'], dtype=object)

We check to see how many unique cancer types there are:
- KIRC
- LUAD
- LUSC
- PRAD
- THCA

Now we will look at how many samples are from each class.

In [8]:
classCount = cancerData['Class'].value_counts()
print(classCount)

Class
KIRC    527
LUAD    510
THCA    501
LUSC    498
PRAD    493
Name: count, dtype: int64
